# Load file

In [2]:
# Load excel file that contains two columns 

#-----------------------------------------------------------
# Source Col            ;           Target Col
# 'Utilities'           ;   'Other manufacturing equipment'
# 'Other stuff'         ;   'Other manufacturing equipment'
#-----------------------------------------------------------

In [3]:
import pandas as pd

In [6]:
PATH_TO_EXCEL = "../data/......"

# Read excel file with header = row number 1
#df = pd.read_excel(PATH_TO_EXCEL, header=0)

# pre-process source and target

In [10]:
# Potential pre-processing steps
# (1) Stemming
# (2) Lemmatization
# (3) Lowercase
# (4) Delete Organizational entities from accounts
# (5) Delete numbers
# (6) Harmonize words
# (7) ...

In [11]:
import spacy

In [101]:
class PreProcessor(object):
    """ Pre-processes a given text string using NLTK and spacy library
    
    """
    
    def __init__(self, text, language="english"):
        """ Loading text and spacy model
        
        :param text: A raw string of words or sentences or corpus
        :param language: Default 'english'. Can be switched to 'german'
        """
        
        self.text = text
        self.language = language
        
        if not ((self.language is "english") or (self.language is "german")):
            raise Exception("Please provide valid language parameter")
            return None
            
    def _text2spacyToken(self, text):
        if self.language is "english":
            nlp = spacy.load("en")
        elif self.language is "german":
            nlp = spacy.load("de")
            
        doc = nlp(text)
        
        return doc.text
        
    def stemming(self, stemmer="SnowballStemmer"):
        
        stemmedText = _text2spacyToken(self.text)
        return stemmedText        
            
    def test(self):
        pass
    

In [102]:
processed = PreProcessor("Hello world", language="german")

In [103]:
txt = processed.stemming

In [104]:
txt

<bound method PreProcessor.stemming of <__main__.PreProcessor object at 0x7f3a8e0da160>>

# Recommender systems

## n-gram (char level) recommender (Jaccard)

## n-gram (word level) recommender (Jaccard)

## TF matching

## TF-IDF matching

## Levenshtein distance

## LSA factorization

## SVM

## word2vec